## Project 2 - 2-mode Network Analysis


An analysis of affiliations between actors and movies in the 2-mode Notre Dame Actor dataset. The main goal of this project is to use clustering techniques such as the island method to try to find small sub-networks of important actors that are frequently collaborating together. In doing so we can also see which movies stand out as focal points for these types of collaborations. 

### Dataset Specs
NDactors.net 
http://vlado.fmf.uni-lj.si/pub/networks/data/ND/NDnets.htm

This is an undirected two-mode network with 520223 vertices (392400 players, 127823 movies) and 1470418 edges; player X plays in movie Y.



In [1]:
#import graphlab as gl
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd

## Data Import

The original dataset was in Pajek format, a wide space seperated table. Some data cleaning was performed on the dataset in R prior to this in order to make sure the table formatting was appropriate. 

Networkx has difficulty handling such a large amount of edge data when it performs projections and transformations, so a sample of 150,000 rows is taken from the main edgelist dataset after import. 


In [2]:
# intitial import and sample
edge_list = pd.read_csv("NDactors_edgelist_melt.csv",sep=",",dtype='string')
edge_list_samp = edge_list.sample(150000,random_state=12345)

In [3]:
actor_nodes = pd.DataFrame(edge_list_samp.actor.unique(),columns=['actor'])
actor_nodes['node_type'] = 'actor'

movie_nodes = pd.DataFrame(edge_list_samp.movie.unique(), columns=['movie'])
movie_nodes['node_type'] = 'movie'


In [4]:
# Build main graph from both node sets and edges as ebunch
# set bipartite attribute to ensure weighted projection will work
a_nodes = list(actor_nodes['actor'])
m_nodes = list(movie_nodes['movie'])
e_bunch = [tuple(i) for i in edge_list_samp.values]

g = nx.Graph()
g.add_nodes_from(a_nodes,node_type='actor', bipartite=0)
g.add_nodes_from(m_nodes,node_type='movie', bipartite=1)
g.add_edges_from(e_bunch)


In [6]:
len(g.nodes())

178996

In [7]:
len(g.edges())

149959

## Weighted Projections and Clustering

The initial graph contains over 178,000 nodes. In order to continue with clustering, we first want to isolate the most important connected subgraph in our large graph. A threshold of 100 is used to find the largest connected subgraph.   


In [157]:
large_sgs = [i for i in nx.connected_component_subgraphs(g) if len(i) > 100]

In [158]:
[len(i) for i in large_sgs] # one large component

[74375]

In [10]:
sg = large_sgs[0] # largest connected subgraph from sample at 74,375

We now perform weighted_projections on this subgraph in order to isolate the two components.

In [11]:
from networkx.algorithms import bipartite as bi

movies,actors = bi.sets(sg)  # split into bipartite sets

In [12]:
## create weighted projections
movie_proj_sg = bi.weighted_projected_graph(sg, movies) 

In [13]:
actor_proj_sg = bi.weighted_projected_graph(sg, actors)

In [14]:
# island method -- probing for thresholds
m = movie_proj_sg.edges(data=True) # split off edge data
a = actor_proj_sg.edges(data=True)

# looking at any weights in the projections that are greater than 1 and 2
len([i for i in a if i[2]['weight'] > 1])

In [16]:
len([i for i in m if i[2]['weight'] > 1])  

247

In [17]:
len([i for i in a if i[2]['weight'] > 2])  

1

In [18]:
len([i for i in m if i[2]['weight'] > 2])  

6

Using a minimum threshold of edge weight 1, we find the nodes with the strongest relationships in both sub-graphs. We now isolate these with the list comprehension using a trim function similar to the one presented in Social Network Analysis Chapter 4. This function returns a a filtered sub-graph at the desired edge-weight threshold. 

Since there aren't a wide range of weights, a trim function that acts as a simple filter suffices. There is no need for stepwise iteration.

In [85]:
def trim(g, weight):
    gtemp = nx.Graph()
    new_ebunch = [i for i in g.edges(data=True) if i[2]['weight'] > weight]    
    gtemp.add_edges_from(new_ebunch)
    return gtemp

In [20]:
act_sg_island =  trim(actor_proj_sg, 1)
mov_sg_island = trim(movie_proj_sg,1)

We now have two island clusters of the projected actors and movies. A quick look at the degree centrality might reveal which nodes are the key players.

In [49]:
## degree centrality measures for the two island clusters
a_sg = nx.degree_centrality(act_sg_island)
m_sg = nx.degree_centrality(mov_sg_island)

pd.DataFrame.from_dict(a_sg,orient='index').sort_values(0,ascending=False).head()

,0
actor_60,0.016509
actor_15844,0.011792
actor_50328,0.009434
actor_9529,0.009434
actor_22255,0.009434


In [50]:
pd.DataFrame.from_dict(m_sg,orient='index').sort_values(0,ascending=False).head()

,0
movie_26395,0.016043
movie_825,0.016043
movie_3543,0.013369
movie_213939,0.013369
movie_274,0.013369


## Most Connected Subgraphs of Weighted Projections

Finally we filter the island clusters into their largest connected components. We can see that actor graph has 6 highly connected components with the largest being 20. The movie graph contains 11 highly connected components with the largest at 36. 


In [37]:
## look at the connected components of new subgraphs
# These give an idea of the actors who in terms of actors

m_cc = [i for i in nx.connected_component_subgraphs(mov_sg_island) if len(i) > 4]
a_cc = [i for i in nx.connected_component_subgraphs(act_sg_island) if len(i) > 4]


In [51]:
[len(i) for i in a_cc]  # 6 large actor components

[20, 8, 6, 6, 6, 5]

In [52]:
[len(i) for i in m_cc] # 11 large movie components

[5, 10, 5, 5, 6, 36, 16, 5, 7, 7, 5]

In [88]:
## combining these into two graphs and then one dataframe for neo
def graph_combine(cc_graphs):
    g = nx.Graph()
    for h in cc_graphs:
        g = nx.compose(g,h)
    return g


In [ ]:
a_islands = graph_combine(a_cc)
m_islands = graph_combine(m_cc)

a_nodes = pd.DataFrame(a_islands.nodes(),columns=['actor_id']).to_csv("actor_nodes.csv")
m_nodes = pd.DataFrame(m_islands.nodes(),columns=['movie_id']).to_csv("movie_nodes.csv")

a_edge_list = pd.DataFrame(a_islands.edges(),columns=['src','trgt']).to_csv("actor_el.csv")
m_edge_list = pd.DataFrame(m_islands.edges(),columns=['src','trgt']).to_csv("movie_el.csv")


<img src="island_clusters.png">

This neo4j chart shows the largest sets of connected components from the island sub-graph. The actors in each of these clusters have the highest degree of collaboration amongst each other in the dataset. We can infer that the connected movies share the most actors and are most likely similar in some way, perhaps the same genre or movie studio. 

## Ego Graph 

Actor 60 played an important role in the above island cluster, being the focal point of the 20 node component in the center of the graph. The actor also had the highest degree centrality, and I thought it would be interesting to build an ego graph from the original two mode network in order to vizualize some of these relationships using a two degree radius. 

In [150]:
# parse nodes and build datframes for ego graph
a60_ego = nx.ego_graph(sg,'actor_60',radius=2)

# parse nodes and build dataframes for output
n = a60_ego.nodes(data=True)
d = {}
d['node_id']=[i[0] for i in n]
d['node_type'] = [i[1]['node_type'] for i in n]

ego_nodes = pd.DataFrame(d).to_csv("ego_nodes.csv")
ego_edges = pd.DataFrame(a60_ego.edges(),columns=['src','trgt']).to_csv("ego_el.csv")


<img src="a60_ego.png">

The red node in middle is Actor 60. In the lower right we have movie 26395 and to the left movie 3543, both of which had the highest centrality scores in their subgraphs. We can see Actor 60 is connected to 16 movies in blue. Several key actors are actually in multiple movies with Actor 60.

One can see from this simple two degree ego graph, the influence that one actor can have on a much larger network. 
